### Author: Celine Combet
### Date Created: 5 Oct 2018
## This example jupyer notebook uses clmm code to generate mock data with a set of configuration parmeters, producing a catalog of ideal and noisy example data.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

### Import mock data module and setup the configuration 

In [ ]:
import clmm.mock_data as mock
from clmm import datatypes

### Mock data generation requires a config dictionary

In [ ]:
config={}
config['cluster_m'] = 1.e15
config['cluster_z'] = 0.3
config['src_z'] = 0.8
config['concentration'] = 4
config['cosmo'] = 'WMAP7-ML'
config['ngals'] = 10000
config['mdef'] = '200c'

ideal_data = mock.MockData(config=config)
noisy_data = mock.MockData(config=config)
noisy_data_z = mock.MockData(config=config)

### Generate the mock catalog from the configuration. Consider 3 examples of increasing complexity:
* Ideal data, no noise, all galaxies at the same redshift
* Noisy data, including shape noise and redshift error, all galaxies at the same redshift
* Noisy data, galaxies following redshift distribution, redshift error, shape noise

In [ ]:
ideal_data.generate()
noisy_data.generate(is_shapenoise=True, shapenoise=0.005, is_zerr=True)
noisy_data_z.generate(is_shapenoise=True, is_zdistribution=True, is_zerr=True)

# Convert the above table into GCData datatype
t_ideal = datatypes.GCData('source catalog', {}, ideal_data.catalog)
t_noisy = datatypes.GCData('source catalog', {}, noisy_data.catalog)
t_noisy_z = datatypes.GCData('source catalog', {}, noisy_data_z.catalog)

#### Ideal catalog first entries: no noise on the shape measurement, all galaxies at z=0.3

In [ ]:
t_ideal.values[0:5]

#### More realistic catalog: noise has been added on the shape measurement, galaxies follow some photoz distribution, gaussian pdf for each photoz is also provided (along with corresponding bins)

In [ ]:
t_noisy_z.values[0:2]

In [ ]:
# Histogram of the redshift distribution of bkg galaxies (starting at z_cluster + 0.1)
hist = plt.hist(t_noisy_z.values['z'], bins=50)

In [ ]:
# pdz for the first galaxy in the catalog
plt.plot(noisy_data_z.catalog['z_bins'][0],noisy_data_z.catalog['z_pdf'][0])

### Compute and plot shear profile

In [ ]:
from clmm import ShearAzimuthalAverager

In [ ]:
cl_dict = {'z':config['cluster_z'], 'ra':0.0, 'dec': 0.0}

saa_ideal = ShearAzimuthalAverager(cl_dict,t_ideal)
saa_noisy = ShearAzimuthalAverager(cl_dict,t_noisy)
saa_noisy_z = ShearAzimuthalAverager(cl_dict,t_noisy_z)


saa_ideal.compute_shear()
saa_noisy.compute_shear()
saa_noisy_z.compute_shear()

saa_ideal.make_shear_profile()
saa_noisy.make_shear_profile()
saa_noisy_z.make_shear_profile()

In [ ]:
# Ideal data
saa_ideal.plot_profile()

In [ ]:
# Noisy data, all galaxies at the same redshift
saa_noisy.plot_profile()

In [ ]:
# Noisy data, galaxies following redshift distribution
saa_noisy_z.plot_profile()